# IMPORTS

In [260]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.model_selection import RandomizedSearchCV

# LOAD DATA

In [261]:
df = pd.read_csv('./ines_df_cleaned.csv')
df.head()

,Unnamed: 0,df_index,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,label
0,0,0,51,1,1,125.0,213.0,0.0,2.0,125.0,1.0,1.4,1.0,1.0,3.0,0.0
1,1,1,54,1,3,120.0,237.0,0.0,0.0,150.0,1.0,1.5,NaN,NaN,7.0,2.0
2,2,2,63,1,4,140.0,0.0,NaN,2.0,149.0,0.0,2.0,1.0,NaN,NaN,2.0
3,3,3,52,0,2,140.0,NaN,0.0,0.0,140.0,0.0,0.0,NaN,NaN,NaN,0.0
4,4,4,55,1,4,140.0,217.0,0.0,0.0,111.0,1.0,5.6,3.0,0.0,7.0,3.0


In [262]:
df.drop(columns= ['Unnamed: 0'], inplace=True)

# IMPUTAR NULOS POR REGRESIÓN

En IterativeImpute, cada característica con un valor perdido se usa como una función de otras características con salida conocida y modela la función para imputaciones.

Luego, el mismo proceso se itera en un bucle para algunas iteraciones y en cada paso, se selecciona una columna de características como salida y y otras columnas de características se tratan como entradas X , luego se ajusta un regresor en (X, y) para y conocidas y se utiliza para predecir los valores perdidos de y . El mismo proceso se repite para cada columna de características en un ciclo y se toma el promedio de todos los valores de regresión múltiple para imputar los valores faltantes para los puntos de datos

In [263]:
label_df = df[['df_index', 'label']]

In [264]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

In [265]:
imp = IterativeImputer(max_iter=10, random_state=0)

In [266]:
imput01 = df[['df_index', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal']]
imput01 = imp.fit_transform(imput01)
imput01 = pd.DataFrame(imput01, columns=['df_index', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal'])
print(imput01.shape)
imput01.head()

(916, 11)


,df_index,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal
0,0.0,125.0,213.000000,0.000000,2.0,125.0,1.0,1.4,1.000000,1.000000,3.000000
1,1.0,120.0,237.000000,0.000000,0.0,150.0,1.0,1.5,1.855420,0.678506,7.000000
2,2.0,140.0,0.000000,0.185656,2.0,149.0,0.0,2.0,1.000000,0.565130,4.848117
3,3.0,140.0,197.034165,0.000000,0.0,140.0,0.0,0.0,1.420978,0.510209,4.331360
4,4.0,140.0,217.000000,0.000000,0.0,111.0,1.0,5.6,3.000000,0.000000,7.000000


In [267]:

df_imput01 = pd.merge(df[['age','sex','cp','label']],imput01,left_index=True, right_index=True)

In [268]:
df_imput01.isna().sum()

age           0
sex           0
cp            0
label       184
df_index      0
trestbps      0
chol          0
fbs           0
restecg       0
thalach       0
exang         0
oldpeak       0
slope         0
ca            0
thal          0
dtype: int64

# DIVIDIR SETS

Set de test: Observaciones en las que la columna 'label' es NaN.

In [269]:
test_ini = df_imput01[df_imput01['label'].isna() == True]
test_ini = test_ini.drop(columns= 'label')
test_ini.reset_index(inplace=False)

,index,age,sex,cp,df_index,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal
0,732,57,1,4,0.0,156.000000,173.000000,0.000000,2.0,119.000000,1.000000,3.000000,3.000000,1.093464,6.317240
1,733,52,1,2,1.0,160.000000,196.000000,0.000000,0.0,165.000000,0.000000,0.000000,1.230772,0.319251,4.022981
2,734,48,1,2,2.0,100.000000,129.344318,0.000000,0.0,100.000000,0.000000,0.000000,1.755526,0.745949,4.907073
3,735,62,1,4,3.0,115.000000,0.000000,0.119212,0.0,128.000000,1.000000,2.500000,3.000000,0.673771,6.384823
4,736,51,1,3,4.0,110.000000,175.000000,0.000000,0.0,123.000000,0.000000,0.600000,1.000000,0.000000,3.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179,911,54,1,4,179.0,200.000000,198.000000,0.000000,0.0,142.000000,1.000000,2.000000,2.000000,0.871371,5.951376
180,912,55,1,2,180.0,110.000000,214.000000,1.000000,1.0,180.000000,0.000000,0.364934,1.402784,0.275832,4.131987
181,913,67,1,3,181.0,152.000000,212.000000,0.000000,2.0,150.000000,0.000000,0.800000,2.000000,0.000000,7.000000
182,914,59,1,1,182.0,170.000000,288.000000,0.000000,2.0,159.000000,0.000000,0.200000,2.000000,0.000000,7.000000


In [270]:
test_ini

,age,sex,cp,df_index,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal
732,57,1,4,0.0,156.000000,173.000000,0.000000,2.0,119.000000,1.000000,3.000000,3.000000,1.093464,6.317240
733,52,1,2,1.0,160.000000,196.000000,0.000000,0.0,165.000000,0.000000,0.000000,1.230772,0.319251,4.022981
734,48,1,2,2.0,100.000000,129.344318,0.000000,0.0,100.000000,0.000000,0.000000,1.755526,0.745949,4.907073
735,62,1,4,3.0,115.000000,0.000000,0.119212,0.0,128.000000,1.000000,2.500000,3.000000,0.673771,6.384823
736,51,1,3,4.0,110.000000,175.000000,0.000000,0.0,123.000000,0.000000,0.600000,1.000000,0.000000,3.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
911,54,1,4,179.0,200.000000,198.000000,0.000000,0.0,142.000000,1.000000,2.000000,2.000000,0.871371,5.951376
912,55,1,2,180.0,110.000000,214.000000,1.000000,1.0,180.000000,0.000000,0.364934,1.402784,0.275832,4.131987
913,67,1,3,181.0,152.000000,212.000000,0.000000,2.0,150.000000,0.000000,0.800000,2.000000,0.000000,7.000000
914,59,1,1,182.0,170.000000,288.000000,0.000000,2.0,159.000000,0.000000,0.200000,2.000000,0.000000,7.000000


Set de entrenamiento y validación: Observaciones en las que la columna 'label' no es NaN.

In [271]:
train_ini = df_imput01[df_imput01['label'].isna() == False]
train_ini.shape
train_ini.reset_index(inplace=False)

,index,age,sex,cp,label,df_index,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal
0,0,51,1,1,0.0,0.0,125.000000,213.000000,0.000000,2.0,125.00000,1.000000,1.40000,1.000000,1.000000,3.000000
1,1,54,1,3,2.0,1.0,120.000000,237.000000,0.000000,0.0,150.00000,1.000000,1.50000,1.855420,0.678506,7.000000
2,2,63,1,4,2.0,2.0,140.000000,0.000000,0.185656,2.0,149.00000,0.000000,2.00000,1.000000,0.565130,4.848117
3,3,52,0,2,0.0,3.0,140.000000,197.034165,0.000000,0.0,140.00000,0.000000,0.00000,1.420978,0.510209,4.331360
4,4,55,1,4,3.0,4.0,140.000000,217.000000,0.000000,0.0,111.00000,1.000000,5.60000,3.000000,0.000000,7.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
727,727,46,1,3,0.0,727.0,120.000000,230.000000,0.000000,0.0,150.00000,0.000000,0.00000,1.403124,0.612396,4.769270
728,728,62,1,3,1.0,728.0,132.960877,204.000000,0.000000,1.0,141.23879,0.352669,0.90105,1.673537,0.833317,5.405075
729,729,43,0,3,0.0,729.0,122.000000,213.000000,0.000000,0.0,165.00000,0.000000,0.20000,2.000000,0.000000,3.000000
730,730,68,1,3,1.0,730.0,150.000000,195.000000,1.000000,0.0,132.00000,0.000000,0.00000,1.595841,0.896790,6.000000


In [272]:
train_ini

,age,sex,cp,label,df_index,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal
0,51,1,1,0.0,0.0,125.000000,213.000000,0.000000,2.0,125.00000,1.000000,1.40000,1.000000,1.000000,3.000000
1,54,1,3,2.0,1.0,120.000000,237.000000,0.000000,0.0,150.00000,1.000000,1.50000,1.855420,0.678506,7.000000
2,63,1,4,2.0,2.0,140.000000,0.000000,0.185656,2.0,149.00000,0.000000,2.00000,1.000000,0.565130,4.848117
3,52,0,2,0.0,3.0,140.000000,197.034165,0.000000,0.0,140.00000,0.000000,0.00000,1.420978,0.510209,4.331360
4,55,1,4,3.0,4.0,140.000000,217.000000,0.000000,0.0,111.00000,1.000000,5.60000,3.000000,0.000000,7.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
727,46,1,3,0.0,727.0,120.000000,230.000000,0.000000,0.0,150.00000,0.000000,0.00000,1.403124,0.612396,4.769270
728,62,1,3,1.0,728.0,132.960877,204.000000,0.000000,1.0,141.23879,0.352669,0.90105,1.673537,0.833317,5.405075
729,43,0,3,0.0,729.0,122.000000,213.000000,0.000000,0.0,165.00000,0.000000,0.20000,2.000000,0.000000,3.000000
730,68,1,3,1.0,730.0,150.000000,195.000000,1.000000,0.0,132.00000,0.000000,0.00000,1.595841,0.896790,6.000000


Separamos características de etiquetas del set de entrenamiento y validación.

In [273]:
# (80% train)
y = train_ini['label']
x= train_ini.drop(columns = 'label')


y_train = y.values
x_train = x.values


print(x_train.shape, y_train.shape)

(732, 14) (732,)


In [274]:
# Split in train and val sets
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.125, random_state=35)

print(x_train.shape, y_train.shape)
print(x_val.shape, y_val.shape)

(640, 14) (640,)
(92, 14) (92,)


Tenemos un split de 70% entrenamiento, 10% validación y 20% test.

Pasamos el set de test a una matriz de vectores

In [275]:
x_test = test_ini.values

### 1.2. Normalizar Datos

In [276]:
def escalar_datos(xtr, xvl, xts):
    '''Escalar datos de entrada (entrenamiento, validación y prueba)
    en el rango de 0 a 1'''
    scaler = MinMaxScaler(feature_range = (0,1))

    # Fit + transform sobre el set de entrenamiento
    xtr_s = scaler.fit_transform(xtr)

    # Transform sobre prueba y validación
    xvl_s = scaler.transform(xvl)
    xts_s = scaler.transform(xts)

    return xtr_s, xvl_s, xts_s

# Ejecutar la función
x_train_s, x_val_s, x_test_s = escalar_datos(x_train, x_val, x_test)

# Verificar
print(x_train_s.min(axis=0),x_train_s.max(axis=0))
print(x_val_s.min(axis=0),x_val_s.max(axis=0))
print(x_test_s.min(axis=0),x_test_s.max(axis=0))

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
[0.125      0.         0.         0.02328767 0.07407407 0.
 0.05395679 0.         0.14084507 0.         0.29545455 0.
 0.05576882 0.        ] [1.         1.         1.         1.00136986 0.72222222 0.69154229
 1.         1.         0.85211268 1.         0.69318182 1.
 0.68525627 0.94138951]
[ 0.02083333  0.          0.          0.         -0.11111111  0.
  0.03291813  0.          0.04929577  0.          0.21590909  0.
  0.05576882  0.        ] [1.02083333 1.         1.         0.25068493 1.         0.67495854
 1.         1.         0.90140845 1.         0.75       1.
 1.         0.94138951]


# 2. MODELOS

## 2.1. Arbol de Decision

In [277]:
def entrenar_y_evaluar_arbol(x_train, y_train, x_val, y_val, x_test):
    
    # Inicializar y entrenar el modelo de árbol de clasificación
    modelo_arbol = DecisionTreeClassifier()
    modelo_arbol.fit(x_train, y_train)

    # Realizar predicciones en el conjunto de validación
    predicciones_val = modelo_arbol.predict(x_val)

    # Calcular la precisión del modelo en el conjunto de validación
    f1_val = f1_score(y_val, predicciones_val, average='weighted')
    print("Precisión del modelo en el conjunto de validación:", f1_val)

    # Realizar predicciones en el conjunto de prueba
    predicciones_test = modelo_arbol.predict(x_test)

    # Convertir los vectores de predicciones y conjuntos de prueba en DataFrames
    df_predicciones_test = pd.DataFrame({'label': predicciones_test}, index=test_ini.index)

    # Agregar las predicciones del conjunto de prueba como una nueva columna en el DataFrame de prueba
    df_predicciones_arbol = pd.concat([test_ini, df_predicciones_test], axis=1)

    return df_predicciones_arbol

In [278]:
df_predicciones_arbol = entrenar_y_evaluar_arbol(x_train_s, y_train, x_val_s, y_val, x_test_s)

Precisión del modelo en el conjunto de validación: 0.4002638626339004


In [279]:
df_predicciones_arbol['ID'] = range(len(df_predicciones_arbol))
df_predicciones_arbol[['ID', 'label']].to_csv('try2_1_data.csv', index=False)

In [280]:
try2_1_df = pd.read_csv('try2_1_data.csv')
try2_1_df['label'].value_counts()

label
0.0    90
1.0    36
3.0    34
2.0    21
4.0     3
Name: count, dtype: int64

## 2.2. KNN

Definir el modelo:

In [281]:
def entrenar_y_evaluar_knn(x_train, y_train, x_val, y_val, x_test, k ):
    
    # Inicializar y entrenar el modelo kNN
    modelo_knn = KNeighborsClassifier(n_neighbors=k)
    modelo_knn.fit(x_train, y_train)

    # Realizar predicciones en el conjunto de validación
    predicciones_val = modelo_knn.predict(x_val)

    # Calcular la precisión del modelo en el conjunto de validación
    f1_val = f1_score(y_val, predicciones_val, average='weighted')
    print("Precisión del modelo en el conjunto de validación:", f1_val)

    # Realizar predicciones en el conjunto de prueba
    predicciones_test = modelo_knn.predict(x_test)

   # Convertir los vectores de predicciones y conjuntos de prueba en DataFrames
    df_predicciones_test = pd.DataFrame({'label': predicciones_test}, index=test_ini.index)

    # Agregar las predicciones del conjunto de prueba como una nueva columna en el DataFrame de prueba
    df_predicciones_knn = pd.concat([test_ini, df_predicciones_test], axis=1)

    return df_predicciones_knn

Hacer un GridSearch para k:

In [282]:
# GridSearch para k

param_grid = {
    'n_neighbors': [3, 5, 7, 9, 11]  # Valores de k a probar
}

# Inicializar el modelo kNN
modelo_knn = KNeighborsClassifier()

# Realizar la búsqueda de hiperparámetros utilizando GridSearchCV con validación cruzada (cv=5)
grid_search = GridSearchCV(modelo_knn, param_grid, cv=5)
grid_search.fit(x_train_s, y_train)

# Obtener el mejor valor de k después de la búsqueda de hiperparámetros
mejor_k = grid_search.best_params_['n_neighbors']
print("Mejor valor de k encontrado:", mejor_k)

Mejor valor de k encontrado: 9


In [283]:
df_predicciones_knn = entrenar_y_evaluar_knn(x_train_s, y_train, x_val_s, y_val, x_test_s, 9)

Precisión del modelo en el conjunto de validación: 0.43123673596373296


In [284]:
df_predicciones_knn

,age,sex,cp,df_index,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,label
732,57,1,4,0.0,156.000000,173.000000,0.000000,2.0,119.000000,1.000000,3.000000,3.000000,1.093464,6.317240,3.0
733,52,1,2,1.0,160.000000,196.000000,0.000000,0.0,165.000000,0.000000,0.000000,1.230772,0.319251,4.022981,0.0
734,48,1,2,2.0,100.000000,129.344318,0.000000,0.0,100.000000,0.000000,0.000000,1.755526,0.745949,4.907073,0.0
735,62,1,4,3.0,115.000000,0.000000,0.119212,0.0,128.000000,1.000000,2.500000,3.000000,0.673771,6.384823,2.0
736,51,1,3,4.0,110.000000,175.000000,0.000000,0.0,123.000000,0.000000,0.600000,1.000000,0.000000,3.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
911,54,1,4,179.0,200.000000,198.000000,0.000000,0.0,142.000000,1.000000,2.000000,2.000000,0.871371,5.951376,2.0
912,55,1,2,180.0,110.000000,214.000000,1.000000,1.0,180.000000,0.000000,0.364934,1.402784,0.275832,4.131987,0.0
913,67,1,3,181.0,152.000000,212.000000,0.000000,2.0,150.000000,0.000000,0.800000,2.000000,0.000000,7.000000,0.0
914,59,1,1,182.0,170.000000,288.000000,0.000000,2.0,159.000000,0.000000,0.200000,2.000000,0.000000,7.000000,0.0


In [285]:
df_predicciones_knn['ID'] = range(len(df_predicciones_knn))
df_predicciones_knn[['ID', 'label']].to_csv('try2_2_data.csv', index=False)

In [286]:
try2_2_df = pd.read_csv('try2_2_data.csv')
try2_2_df['label'].value_counts()

label
0.0    96
1.0    43
2.0    26
3.0    19
Name: count, dtype: int64

## 2.2.2 knn con otro gridsearch

In [287]:
param_grid2 = {
    'n_neighbors': list(range(1, 21))  # Valores de k a probar
}

# Inicializar el modelo kNN
modelo_knn2 = KNeighborsClassifier()

# Realizar la búsqueda de hiperparámetros utilizando GridSearchCV con validación cruzada (cv=5)
grid_search2 = GridSearchCV(modelo_knn2, param_grid2, cv=5)
grid_search2.fit(x_train_s, y_train)

# Obtener el mejor valor de k después de la búsqueda de hiperparámetros
mejor_k2 = grid_search2.best_params_['n_neighbors']
print("Mejor valor de k encontrado:", mejor_k2)

Mejor valor de k encontrado: 9


In [288]:
df_predicciones_knn2 = entrenar_y_evaluar_knn(x_train_s, y_train, x_val_s, y_val, x_test_s, 4)

Precisión del modelo en el conjunto de validación: 0.4973919351513467


In [289]:
df_predicciones_knn2['ID'] = range(len(df_predicciones_knn2))
df_predicciones_knn2[['ID', 'label']].to_csv('try2_3_data.csv', index=False)

In [290]:
try2_3_df = pd.read_csv('try2_3_data.csv')
try2_3_df['label'].value_counts()

label
0.0    100
2.0     33
1.0     29
3.0     20
4.0      2
Name: count, dtype: int64

## 2.3. SVM

In [291]:
param_grid = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf', 'poly', 'sigmoid'],
}

# Inicializar el modelo SVM
modelo_svm = SVC()

# Realizar la búsqueda de hiperparámetros utilizando RandomSearchCV con validación cruzada (cv=5)
grid_search = GridSearchCV(modelo_svm, param_grid)
grid_search.fit(x_train_s, y_train)

# Obtener los mejores hiperparámetros después de la búsqueda de hiperparámetros
mejores_hiperparametros = grid_search.best_params_
print("Mejores hiperparámetros encontrados:", mejores_hiperparametros)

Mejores hiperparámetros encontrados: {'C': 10, 'kernel': 'linear'}


In [292]:
def entrenar_y_evaluar_svm(x_train, y_train, x_val, y_val, x_test, C, kernel):
    
    # Inicializar y entrenar el modelo SVM
    modelo_svm = SVC(C=C, kernel=kernel)
    modelo_svm.fit(x_train, y_train)

    # Realizar predicciones en el conjunto de validación
    predicciones_val = modelo_svm.predict(x_val)

    # Calcular la precisión del modelo en el conjunto de validación
    f1_val = f1_score(y_val, predicciones_val, average = 'weighted')
    print("Precisión del modelo en el conjunto de validación:", f1_val)

    # Realizar predicciones en el conjunto de prueba
    predicciones_test = modelo_svm.predict(x_test)

   # Convertir los vectores de predicciones y conjuntos de prueba en DataFrames
    df_predicciones_test = pd.DataFrame({'label': predicciones_test}, index=test_ini.index)

    # Agregar las predicciones del conjunto de prueba como una nueva columna en el DataFrame de prueba
    df_predicciones_svm = pd.concat([test_ini, df_predicciones_test], axis=1)

    return df_predicciones_svm

In [293]:
df_predicciones_svm = entrenar_y_evaluar_svm(x_train_s, y_train, x_val_s, y_val, x_test_s, 10, 'linear')

Precisión del modelo en el conjunto de validación: 0.4936275222609038


In [294]:
df_predicciones_svm['ID'] = range(len(df_predicciones_svm))
df_predicciones_svm[['ID', 'label']].to_csv('try2_4_data.csv', index=False)

In [295]:
try2_4_df = pd.read_csv('try2_4_data.csv')
try2_4_df['label'].value_counts()

label
0.0    93
1.0    43
2.0    31
3.0    17
Name: count, dtype: int64